In [ ]:
#This is a hack to make the %%cpp work inside mybinder.org - ignore
from ROOT import TObject

In [ ]:
%%cpp
//Create a c++ object representing a LorentzVector
//More details at https://root.cern.ch/doc/master/classTLorentzVector.html
TLorentzVector v;
//Print the vectors components - should see all components initialised to zero
v.Print()

In [ ]:
%%cpp
//We see this does not work, because 1 and 2 are integers.
//The template arguments to DeltaPhi must be classes that implement specific methods.
double test = ROOT::Math::VectorUtil::DeltaPhi(1,2)

In [ ]:
%%cpp
//We try deltaphi again, this time with instances of classes that do implement the required methods.
TLorentzVector mark1(0.0,0.0,0.0,0.0);
TLorentzVector mark2(0.0,0.0,0.0,0.0);
double test = ROOT::Math::VectorUtil::DeltaPhi(mark1,mark2);
test

In [ ]:
#Now do the same in python
#Don't forget in python we must import each ROOT C++ class we want to use.
from ROOT import TLorentzVector
mark1 = TLorentzVector(0.0,0.0,0.0,0.0)
mark2 = TLorentzVector(0.0,0.0,0.0,0.0)
from ROOT import Math
#Won't work
Math.VectorUtil.DeltaPhi(1,2)

In [ ]:
#Will work
Math.VectorUtil.DeltaPhi(mark1,mark2)

In [ ]:
#Task 1
#Download https://nbviewer.jupyter.org/url/root.cern/doc/master/notebooks/mathcoreVectorCollection.C.nbconvert.ipynb
#and run it.
#Note I spotted the last two cells are missing the %%cpp!

In [ ]:
%%cpp
//Magic numbers
//No need to write "double M = 0.13957;" in above example
TDatabasePDG db;
//If everyone uses it, all code uses consistent numbers in calculations - easier to debug your code
//with respect to colleagues codes etc
db.GetParticle(211)->Mass()

In [ ]:
%%cpp
//Roofit in C++
//Each variable, RooRealVar, has a name, title and range
//mes has the name "mes" and title "m_{ES} (GeV)" - we specify the range to be in 5.20 and 5.30
//the initial value of the variable will be in the mid-point of that range
RooRealVar mes("mes","m_{ES} (GeV)",5.20,5.30);
//We can also specify the initial value of a variable - so for sigmean we specify 5.28 and again 
//a range between 5.20 and 5.30
RooRealVar sigmean("sigmean","B^{#pm} mass",5.28,5.20,5.30);
RooRealVar sigwidth("sigwidth","B^{#pm} width",0.0027,0.001,1.);
//Create a PDF of a Gaussian shape using mes as our data with a mean and width of sigmean and sigwitdh
RooGaussian gauss("gauss","gaussian PDF",mes,sigmean,sigwidth);

In [ ]:
%%cpp
//Now lets proceed to draw our PDF
//First we get a pointer to a RooPlot object, from the RooRealVar mes
RooPlot* frame = mes.frame();
//Ten we plot our PDF, gauss, onto that RooPlotObject
gauss.plotOn(frame);
//Then as usual we can create a TCanvas and draw both the RooPlot object and the canvas.
TCanvas rooCan;
frame->Draw();
rooCan.Draw()

In [4]:
#Roofit example, this time in python
#Again note the main difference is that we need to import each c++ class explicitly,
#and that variable declaration has a different syntax.
#Also note we don't worry about whether a variable is a pointer or not - the "->" and "." syntax in c++
#Here, in python, we just always use "."
from ROOT import RooRealVar
mes = RooRealVar("mes","m_{ES} (GeV)",5.20,5.30)
sigmean = RooRealVar("sigmean","B^{#pm} mass",5.28,5.20,5.30)
sigwidth = RooRealVar("sigwidth","B^{#pm} width",0.0027,0.001,1.)
from ROOT import RooGaussian
gauss = RooGaussian("gauss","gaussian PDF",mes,sigmean,sigwidth)

In [5]:
frame = mes.frame()
gauss.plotOn(frame)
from ROOT import TCanvas
rooCan = TCanvas()
frame.Draw()
rooCan.Draw()

In [6]:
#Task2
#Create a Landau PDF, along with appropriate parameters and repeat the above workflow shown for the Gaussian case.

In [7]:
#Roofit example with signal and background shapes
argpar = RooRealVar("argpar","argus shape parameter",-20.0,-100.,-1.)
#This time we also create a parameter with a fixed value using the RooConstVar class
from ROOT import RooConstVar
argconst = RooConstVar("argconst","argus constant",5.291)
#Our background shape will be an Argus function
from ROOT import RooArgusBG
argus = RooArgusBG("argus","Argus PDF",mes,argconst,argpar)
#Eventually we will combine our background and signal shapes
#So we create variables to weight those shapes
nsig = RooRealVar("nsig","#signal events",200,0.,10000)
nbkg = RooRealVar("nbkg","#background events",800,0.,10000) 
from ROOT import RooAddPdf,RooArgList
#Finally we create the full shape which is the sum of a background (Argus) and signal (Gaussian)
#The background and signal are weighted by nbkg and nsig
sum = RooAddPdf("sum","g+a",RooArgList(gauss,argus),RooArgList(nsig,nbkg))

In [8]:
from ROOT import RooArgSet
#The generate function allows us to generate some Monte Carlo ("toy Monte Carlo") events using the PDF shape
data = sum.generate(RooArgSet(mes),2000)
#Then we can fit the shape to that toy MC data (uses extended ML fit)
sum.fitTo(data)
mesframe = mes.frame()
from ROOT import RooFit,kDashed
#We plot three things - the toy MC data, the full background + signal fitted shape, and the Argus fitted shape
#Try commenting them out, and doing it one by one to see what they do!
data.plotOn(mesframe)
sum.plotOn(mesframe)
sum.plotOn(mesframe,RooFit.Components("argus"),RooFit.LineStyle(kDashed))
rooCan.Clear()
mesframe.Draw()
#Hint - scroll down to see the plot!
rooCan.Draw()

[#1] INFO:Minimization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Fitting -- RooAbsPdf::fitTo(sum) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#1] INFO:Fitting -- Creation of NLL object took 3.31571 ms
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_sum_sumData) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
Minuit2Minimizer: Minimize with max-calls 2500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = -18178.183919428815
Edm   = 7.61294724854681112e-05
Nfcn  = 112
argpar	  = -22.9054	 +/-  3.42143	(limited)
nbkg	  = 1612.74	 +/-  44.6723	(limited)
nsig	  = 387.37	 +/-  27.7674	(limited)
sigmean	  = 5.27987	 +/-  0.000215769	(limited)
sigwidth	  = 0.00300965	 +/-  0.000199023	

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 1.11175 ms
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -17788.81972 Edm =       387.8769989 NCalls =     23
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : -17788.81972
  Edm           : 387.8769989
  Internal parameters:	[     0.6638599611    -0.9972832224     -1.287002218     0.6435011088     -1.488269535]	
  Internal gradient  :	[      14.82946743     -2757.194664     -1308.655031       126.275939     -596.9436515]	
  Internal covariance matrix:
[[    0.012706854              0              0              0              0]
 [              0  0.00015334155              0              0              0]
 [              0              0  0.00021750069              0              0]
 [              0              0              0  4.9189718e-05              0]
 [              0              0      

In [9]:
from ROOT import TFile
#Lets get the tree2 file again
filePython = TFile("../Lecture1/tree2.root")
#define a variable with an appropriate range
destep = RooRealVar("destep","destep",15e-06,25e-06)
from ROOT import RooDataSet
#Get the TTree
t2 = filePython.Get("t2;1")
#Then we can generate a RooDataset from the TTree and the variable destep
data = RooDataSet("data","data",RooArgSet(destep),Import=t2)

[#1] INFO:DataHandling -- RooAbsReal::attachToTree(destep) TTree Float_t branch destep will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #0 because destep cannot accommodate the value 0
[#0] WARNING:DataHandling -- RooTreeDataStore::loadValues(data) Ignored 1 out-of-range events


In [ ]:
#Get a frame from our variable and plot the data on it
newFrame = destep.frame()
data.plotOn(newFrame)
#Then draw the fram on our canvas
newFrame.Draw()
rooCan.Draw()

In [ ]:
#Task3
#Set up a PDF shape and use it to fit the destep data.
#What kind of shape works well?
#Try it again using the native ROOT fit functionality we saw in previous lectures to compare the methodologies.